In [1]:
from importlib import reload

In [2]:
import statsmodels.api as sm

In [3]:
from datetime import datetime, timedelta

import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt

from IPython.display import Image, display

from dowhy import CausalModel

In [4]:
pd.set_option("mode.chained_assignment", None)

In [8]:
import util.load as load_util
import util.explore as explore_util
import util.experiment as experiment

In [9]:
wdi_df, series_df = load_util.load_wdi()
project_df = load_util.load_projects()
wb_df = load_util.extract_wb_projects(project_df)

## Constructing time gaps

In [11]:
sector_aggs = project_df.groupby('sector').agg(
    number_projects=('ppd_project_id', 'nunique'),
    earliest_year=('start_dt', 'min'),
    latest_project=('start_dt', 'max'),
    number_donors=('donor_name', 'nunique'),
    avg_rating=('six_overall_rating', 'mean'),
    avg_duration=('project_duration', 'mean')
)

In [12]:
feature_cols = [
    'country_code', 
    'donor_name', 
    'aiddata_sectorname', 
    'six_overall_rating', 
    'start_date', 
    'completion_date', 
    'project_duration',
    'sector'
]

pdf = load_util.narrow_convert_project_data(project_df, feature_cols)

total_countries = pdf.country_code.nunique()
earliest_year = pdf.start_year.min()
latest_year = pdf.start_year.max()
year_range = range(int(earliest_year), int(latest_year))

## First approach: binary treatment, replicating papers on causal structure

### Assemble education panel

In [10]:
# load in panel assembled by DG, and country code cross-matches

In [14]:
reload(experiment)

<module 'util.experiment' from '/home/luke/newproj/wbwdi/src/util/experiment.py'>

In [12]:
# for each country-year, calculate mean growth in education indicators at year + lag

In [15]:
df, tdf = explore_util.assemble_sectoral_df(
    pdf, 
    experiment.sector_outcome_indicators['education'],
    year_range,
    interpolate_limit=5,
    persisted_lag_table='../data/transformed_data/education_df.csv'
)

In [33]:
if not 'ppd_countrycode' in df.columns:
    df = df.merge(ccode_corr[['ppd_countrycode', 'wdi_countrycode', 'wdi_countryname']], left_on='country',
                   right_on='ppd_countrycode', how='left')
    df['country'] = df['wdi_countrycode']
    df = df[df['country'].notna()].drop(columns=['wdi_countrycode'])

In [34]:
df.head()

,year,country,ppd_countrycode,wdi_countryname,project_start_year,project_completed_year,education_lag_1_growth,education_lag_1_count,education_lag_2_growth,education_lag_2_count,...,education_lag_6_growth,education_lag_6_count,education_lag_7_growth,education_lag_7_count,education_lag_8_growth,education_lag_8_count,education_lag_9_growth,education_lag_9_count,education_lag_-4_growth,education_lag_-4_count
0,1956,CHN,CHN,China,False,False,NaN,0,NaN,0,...,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0
1,1956,PHL,PHI,Philippines,False,False,NaN,0,NaN,0,...,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0
2,1956,NPL,NEP,Nepal,False,False,NaN,0,NaN,0,...,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0
3,1956,LKA,SRI,Sri Lanka,False,False,NaN,0,NaN,0,...,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0
4,1956,TON,TON,Tonga,False,False,NaN,0,NaN,0,...,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0


In [35]:
ddf.head()

,year,countrycode,countryname,regionname,fcv_ind,lendingtype,incomelevel,pc_commit_total,pc_commit_education,pc_commit_health,...,life_expectancy,hiv_prevalence,health_share_gov_exp,cash_surplus_deficit,access_water,access_sanitation,political_rights,civil_liberties,conflict,freedom_house
0,1990,AFG,Afghanistan,South Asia,FCS,IDA,LIC,11.130618,1.326267,2.602422,...,50.331001,0.1,NaN,NaN,NaN,NaN,7.0,7.0,1,7.0
1,1991,AFG,Afghanistan,South Asia,FCS,IDA,LIC,8.588026,1.464879,1.493611,...,50.999001,0.1,NaN,NaN,21.2,20.5,7.0,7.0,1,7.0
2,1992,AFG,Afghanistan,South Asia,FCS,IDA,LIC,8.644200,1.124778,0.535377,...,51.640999,0.1,NaN,NaN,21.3,20.5,6.0,6.0,1,6.0
3,1993,AFG,Afghanistan,South Asia,FCS,IDA,LIC,5.814673,0.598482,0.596612,...,52.256001,0.1,NaN,NaN,21.4,20.5,7.0,7.0,1,7.0
4,1994,AFG,Afghanistan,South Asia,FCS,IDA,LIC,4.026480,0.081287,0.098373,...,52.841999,0.1,NaN,NaN,21.4,20.6,7.0,7.0,1,7.0


In [36]:
edu_proj, df = futil.assemble_sector_proj_df(project_df, 'Education', df)
df = futil.construct_sector_lag_table(df, tdf, pdf, education_outcome_indicators, 'Education')

In [38]:
# then: add in the various confounders, treatments, controlsc

In [39]:
ddf.head()

,year,countrycode,countryname,regionname,fcv_ind,lendingtype,incomelevel,pc_commit_total,pc_commit_education,pc_commit_health,...,life_expectancy,hiv_prevalence,health_share_gov_exp,cash_surplus_deficit,access_water,access_sanitation,political_rights,civil_liberties,conflict,freedom_house
0,1990,AFG,Afghanistan,South Asia,FCS,IDA,LIC,11.130618,1.326267,2.602422,...,50.331001,0.1,NaN,NaN,NaN,NaN,7.0,7.0,1,7.0
1,1991,AFG,Afghanistan,South Asia,FCS,IDA,LIC,8.588026,1.464879,1.493611,...,50.999001,0.1,NaN,NaN,21.2,20.5,7.0,7.0,1,7.0
2,1992,AFG,Afghanistan,South Asia,FCS,IDA,LIC,8.644200,1.124778,0.535377,...,51.640999,0.1,NaN,NaN,21.3,20.5,6.0,6.0,1,6.0
3,1993,AFG,Afghanistan,South Asia,FCS,IDA,LIC,5.814673,0.598482,0.596612,...,52.256001,0.1,NaN,NaN,21.4,20.5,7.0,7.0,1,7.0
4,1994,AFG,Afghanistan,South Asia,FCS,IDA,LIC,4.026480,0.081287,0.098373,...,52.841999,0.1,NaN,NaN,21.4,20.6,7.0,7.0,1,7.0


In [40]:
df = df.merge(ddf, how='left', left_on=['country', 'year'], right_on=['countrycode', 'year'])

In [41]:
df.tail()

,year,country,ppd_countrycode,wdi_countryname,project_start_year,project_completed_year,education_lag_1_growth,education_lag_1_count,education_lag_2_growth,education_lag_2_count,...,life_expectancy,hiv_prevalence,health_share_gov_exp,cash_surplus_deficit,access_water,access_sanitation,political_rights,civil_liberties,conflict,freedom_house
9459,2011,GNQ,EQG,Equatorial Guinea,False,False,1.021988,3,1.006549,3,...,55.945000,5.9,1.472929,NaN,47.6,75.4,7.0,7.0,0.0,7.0
9460,2011,SUR,SUR,Suriname,False,False,0.992491,3,0.986839,3,...,70.676003,1.4,8.512039,-0.786504,94.4,79.4,2.0,2.0,0.0,2.0
9461,2011,SSD,SSD,South Sudan,False,False,0.973313,2,0.946626,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9462,2011,CUB,CUB,Cuba,False,False,0.993128,3,0.991756,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9463,2011,USA,USA,United States,False,False,1.001580,1,0.998160,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
# first : country rating
def find_latest_govt_rating(country, year):
    country_proj = project_df[
        (project_df['country_code'] == country) 
        & (project_df['start_year'] <= year)
        & (project_df['wb_government_partner_rating'].notna())]
    if len(country_proj) == 0:
        return np.nan
    else:
        proj_year = country_proj[country_proj.start_year == country_proj.start_year.max()]
        return proj_year['wb_government_partner_rating'].sort_values(ascending=False).iloc[0]

In [58]:
add_most_recent_rating = False

if add_most_recent_rating and 'most_recent_govt_rating' not in df:
    df['most_recent_govt_rating'] = df.apply(lambda row: find_latest_govt_rating(row['country'], row['year']), axis=1)
    df.most_recent_govt_rating.value_counts()

## Check for missing values

In [45]:
panel_non_data = ['year', 'countrycode', 'regionname', 'fcv_ind', 'lendingtype', 'incomelevel']
non_data_cols = ['year', 'country', 'ppd_countrycode', 'wdi_countryname', 'project_completed_year'] + panel_non_data
data_cols = [col for col in df.columns if col not in non_data_cols]
ddf_data_cols = [col for col in ddf.columns if col not in panel_non_data]

In [46]:
def extract_culprit_counts(df, data_cols):
    null_df = df.isna()
    null_df['number_missing'] = null_df[data_cols].sum(axis=1)
    clean_number_base = null_df['number_missing'].value_counts().to_dict()[0]
    culprit_counts = {}
    for test_col in data_cols:
        tmp_cols = [col for col in data_cols if col != test_col]
        temp_df = null_df.drop(columns=[test_col])
        temp_df['number_missing'] = temp_df[tmp_cols].sum(axis=1)
        new_clean = temp_df['number_missing'].value_counts().to_dict()[0]
        culprit_counts[test_col] = new_clean - clean_number_base
        
    return culprit_counts, null_df

In [54]:
exclude_govt_rating = True # empty in a lot of places

if exclude_govt_rating and 'most_recent_govt_rating' in df.columns:
    df = df.drop(columns=['most_recent_govt_rating'])

if exclude_govt_rating and 'most_recent_govt_rating' in data_cols:
    data_cols.remove('most_recent_govt_rating')

In [56]:
culprit_counts, null_df = extract_culprit_counts(df, data_cols)
culprit_counts

{'project_start_year': 0,
 'education_lag_1_growth': 0,
 'education_lag_1_count': 0,
 'education_lag_2_growth': 0,
 'education_lag_2_count': 0,
 'education_lag_3_growth': 0,
 'education_lag_3_count': 0,
 'education_lag_4_growth': 0,
 'education_lag_4_count': 0,
 'education_lag_5_growth': 0,
 'education_lag_5_count': 0,
 'education_lag_6_growth': 0,
 'education_lag_6_count': 0,
 'education_lag_7_growth': 0,
 'education_lag_7_count': 0,
 'education_lag_8_growth': 0,
 'education_lag_8_count': 0,
 'education_lag_9_growth': 45,
 'education_lag_9_count': 0,
 'education_lag_-4_growth': 6,
 'education_lag_-4_count': 0,
 'countryname': 0,
 'pc_commit_total': 0,
 'pc_commit_education': 0,
 'pc_commit_health': 0,
 'pc_commit_wash': 0,
 'commit_total': 0,
 'commit_education': 0,
 'commit_health': 0,
 'commit_wash': 0,
 'population': 0,
 'young_population': 0,
 'fertility': 0,
 'oda_share_gni': 0,
 'oda_pc': 0,
 'gdp_pc': 0,
 'gdp_pc_ppp': 0,
 'inflation': 7,
 'trade_share_gdp': 18,
 'edu_ner': 0,


In [60]:
ddf_culprits, ddf_nulls = extract_culprit_counts(ddf, ddf_data_cols)
ddf_culprits

{'countryname': 0,
 'pc_commit_total': 0,
 'pc_commit_education': 0,
 'pc_commit_health': 0,
 'pc_commit_wash': 0,
 'commit_total': 0,
 'commit_education': 0,
 'commit_health': 0,
 'commit_wash': 0,
 'population': 0,
 'young_population': 0,
 'fertility': 0,
 'oda_share_gni': 0,
 'oda_pc': 0,
 'gdp_pc': 0,
 'gdp_pc_ppp': 0,
 'inflation': 14,
 'trade_share_gdp': 19,
 'edu_ner': 0,
 'edu_aner': 1,
 'edu_completion': 24,
 'edu_share_gov_exp': 37,
 'edu_pupil_teacher': 7,
 'female_adult_literacy': 0,
 'adult_literacy': 0,
 'physicians_rate': 8,
 'mortality_under5': 0,
 'immunization': 0,
 'life_expectancy': 0,
 'hiv_prevalence': 83,
 'health_share_gov_exp': 53,
 'cash_surplus_deficit': 214,
 'access_water': 0,
 'access_sanitation': 0,
 'political_rights': 0,
 'civil_liberties': 0,
 'conflict': 0,
 'freedom_house': 0}

*Note*: There is no surplus deficit in the standard WDI, so using net borrowing in year (is available)

In [64]:
save_df = False

if save_df:
    df.to_csv('../data/transformed_data/education_df.csv', index=False)

## Some vanilla OLS

In [65]:
import statsmodels.api as sm

In [66]:
df.head()

,year,country,ppd_countrycode,wdi_countryname,project_start_year,project_completed_year,education_lag_1_growth,education_lag_1_count,education_lag_2_growth,education_lag_2_count,...,life_expectancy,hiv_prevalence,health_share_gov_exp,cash_surplus_deficit,access_water,access_sanitation,political_rights,civil_liberties,conflict,freedom_house
0,1956,CHN,CHN,China,False,False,NaN,0,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1956,PHL,PHI,Philippines,False,False,NaN,0,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1956,NPL,NEP,Nepal,False,False,NaN,0,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1956,LKA,SRI,Sri Lanka,False,False,NaN,0,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1956,TON,TON,Tonga,False,False,NaN,0,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
df.isna().sum() / len(df)

year                  0.000000
country               0.000000
ppd_countrycode       0.000000
wdi_countryname       0.000000
project_start_year    0.000000
                        ...   
access_sanitation     0.703614
political_rights      0.694421
civil_liberties       0.694421
conflict              0.690828
freedom_house         0.694421
Length: 69, dtype: float64

In [68]:
# Adding categorical variables for country, for replication purposes, although means dimensionality explosion (for unclear gain)

Education: Specification 2
```
regress  last_ner <- first_ner pc_commit_education [per capita commitment amount=
        edu_share_gov_exp edu_pupil_teacher young_population
        gdp_pc_ppp cash_surplus_deficit inflation trade_share_gdp
        freedom_house i.period i.ncountrycode if countrytoinclude == 1, r
```

In [69]:
df.columns

Index(['year', 'country', 'ppd_countrycode', 'wdi_countryname',
       'project_start_year', 'project_completed_year',
       'education_lag_1_growth', 'education_lag_1_count',
       'education_lag_2_growth', 'education_lag_2_count',
       'education_lag_3_growth', 'education_lag_3_count',
       'education_lag_4_growth', 'education_lag_4_count',
       'education_lag_5_growth', 'education_lag_5_count',
       'education_lag_6_growth', 'education_lag_6_count',
       'education_lag_7_growth', 'education_lag_7_count',
       'education_lag_8_growth', 'education_lag_8_count',
       'education_lag_9_growth', 'education_lag_9_count',
       'education_lag_-4_growth', 'education_lag_-4_count', 'countrycode',
       'countryname', 'regionname', 'fcv_ind', 'lendingtype', 'incomelevel',
       'pc_commit_total', 'pc_commit_education', 'pc_commit_health',
       'pc_commit_wash', 'commit_total', 'commit_education', 'commit_health',
       'commit_wash', 'population', 'young_population', 'fer

In [78]:
cols_for_replication = ['education_lag_4_growth', 'country', 'project_completed_year', 
                        'pc_commit_education', 'edu_pupil_teacher', 'young_population',
                        'gdp_pc_ppp', 'cash_surplus_deficit', 'inflation', 'trade_share_gdp',
                        'freedom_house']

target_col = 'education_lag_4_growth'

In [150]:
add_country_feffects = False

ols_df = df[cols_for_replication]

if add_country_feffects:
    ols_df = pd.concat((ols_df, pd.get_dummies(df['country'], drop_first=True)), axis=1)

ols_df.head()

,education_lag_4_growth,country,project_completed_year,pc_commit_education,edu_pupil_teacher,young_population,gdp_pc_ppp,cash_surplus_deficit,inflation,trade_share_gdp,freedom_house
0,NaN,CHN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,PHL,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NPL,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,LKA,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,TON,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [151]:
len(ols_df)

In [152]:
data = ols_df.dropna()
len(data)

In [153]:
category_cols = [target_col, 'country', 'year', 'ppd_countrycode', 'wdi_countryname', 'project_start_year']
y = data[target_col]
x = data.drop(columns=category_cols, errors='ignore').replace({False: 0, True: 1})

In [154]:
x.head()

,project_completed_year,pc_commit_education,edu_pupil_teacher,young_population,gdp_pc_ppp,cash_surplus_deficit,inflation,trade_share_gdp,freedom_house
5747,0,6.314927,32.667149,40.935684,4231.7925,-2.811081,12.177352,60.800270,3.0
5751,0,3.835502,31.682180,42.064705,2436.5964,-2.173136,6.953361,89.571548,2.5
5760,1,4.119413,23.271490,36.451164,4532.5400,0.401202,7.819191,52.891861,5.5
5796,0,43.155426,31.656540,44.966103,9074.2061,19.084702,11.396346,104.847320,1.5
5818,0,6.918624,51.484001,46.450027,3222.9446,-5.585779,1.099381,37.496445,6.0


In [155]:
est = sm.OLS(y, sm.add_constant(x)).fit()

In [156]:
print(est.summary())

                              OLS Regression Results                              
Dep. Variable:     education_lag_4_growth   R-squared:                       0.058
Model:                                OLS   Adj. R-squared:                  0.051
Method:                     Least Squares   F-statistic:                     7.585
Date:                    Thu, 03 Jun 2021   Prob (F-statistic):           7.55e-11
Time:                            21:24:20   Log-Likelihood:                 1141.2
No. Observations:                    1115   AIC:                            -2262.
Df Residuals:                        1105   BIC:                            -2212.
Df Model:                               9                                         
Covariance Type:                nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------

In [157]:
save_to_csv = True

if save_to_csv:
    with open('./current_ols_no_fe.txt', 'w') as file:
        file.write(est.summary().as_text())

# Execute do-why

In [ ]:
df['sector_delta_prior'] = df['education_lag_-4_growth']
df['sector_delta_post'] = df['education_lag_4_growth']

In [ ]:
sectoral_effect_graph_binary = """digraph { 
    "Govt-Quality"[latent];
    project_completed_year [exposure, label="project_completed"];
    sector_delta_prior [adjusted, label="sector_outcome_growth_prior"];
    sector_delta_post [outcome, label=sector_outcome_growth_lagged];
    most_recent_govt_rating [label="WBGovtRating"];
    "Govt-Quality" -> project_completed_year;
    "Govt-Quality" -> sector_delta_prior;
    "Govt-Quality" -> most_recent_govt_rating;
    project_completed_year -> sector_delta_post;
    sector_delta_prior -> sector_delta_post;
    most_recent_govt_rating -> project_completed_year;
    }"""

In [ ]:
causal_columns = ['sector_delta_prior', 'most_recent_govt_rating', 'project_completed_year', 'sector_delta_post']

In [ ]:
causal_df = df[causal_columns]
pre_drop_N = len(causal_df)
causal_df = causal_df.dropna()
post_drop_N = len(causal_df)

print("N pre NA drop: ", pre_drop_N, " and post: ", post_drop_N)

In [ ]:
causal_df.project_completed_year.value_counts()

In [ ]:
model = CausalModel(
    data=causal_df,
    graph=sectoral_effect_graph_binary.replace("\n", " "),
    treatment="project_completed_year",
    outcome="sector_delta_post"
)

model.view_model()
display(Image(filename="causal_model.png"))

In [ ]:
identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)
print(identified_estimand)

In [ ]:
estimate = model.estimate_effect(identified_estimand, method_name="backdoor.linear_regression",
                                 target_units="ate", test_significance=True)
print(estimate)

In [ ]:
causal_df.head()

In [ ]:
estimator_list = ["backdoor.propensity_score_matching", "backdoor.propensity_score_weighting", "backdoor.causalml.inference.meta.LRSRegressor"]
method_params= [ None, None, { "init_params":{} } ]

In [ ]:
refuter_list = ["bootstrap_refuter", "data_subset_refuter"]

In [ ]:
# for estimator, method_params in zip(estimator_list, method_params):
#     estimate = model.estimate_effect(identified_estimand, method_name=estimator, method_params=method_params)

In [ ]:
refuter = model.refute_estimate(identified_estimand, estimate, method_name="bootstrap_refuter")

In [ ]:
print(refuter)

In [ ]:
run_refuter = lambda refuter_name, refuter_params: model.refute_estimate(identified_estimand, estimate, method_name=refuter_name, **refuter_params)

In [ ]:
# c_cause_refuter = model.refute_estimate(identified_estimand, estimate, method_name="random_common_cause")
c_cause_refuter = run_refuter("random_common_cause", {})

In [ ]:
print(c_cause_refuter)

In [ ]:
placebo_refuter = run_refuter("placebo_treatment_refuter", dict(placebo_type="permuter"))

In [ ]:
print(placebo_refuter)

In [ ]:
sample_refuter = run_refuter("data_subset_refuter", dict(subset_fraction=0.9))

In [ ]:
print(sample_refuter)

In [ ]:
# Note: took a bit of digging, but the "p value" here relates to a null hypothesis that 
# the estimate obtained is part of the general set of distributions, i.e., we want this to hold
# (since it suggests that the family of estimators is not significantly affected by subsampling)

In [ ]:
estimate.interpret(method_name="textual_effect_interpreter")

In [ ]:
bootstrap_refuter = run_refuter("bootstrap_refuter", {})

In [ ]:
print(bootstrap_refuter)

In [ ]:
# cribbed for now from example notebook
dummy_args = {'num_simulations':5,'transformation_list': [('random_forest',{'n_estimators':100, 'max_depth':6})], 'true_causal_effect':(lambda x:0.5)}

In [ ]:
dummy_refuter = run_refuter("dummy_outcome_refuter", dummy_args)

In [ ]:
print(dummy_refuter[0])

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LassoCV

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
# cribbed for now
dml_params = { 
    "init_params": { 
        'model_y':GradientBoostingRegressor(),
        'model_t': GradientBoostingRegressor(),
        "model_final":LassoCV(fit_intercept=False),
        'featurizer':PolynomialFeatures(degree=1, include_bias=True)
    },
    "fit_params": {}
}

In [ ]:
estimate = model.estimate_effect(identified_estimand, method_name="backdoor.econml.dml.DML", method_params=dml_params,
                                 target_units="ate", test_significance=True)
print(estimate)

In [ ]:
print(estimate)